# Task #2: Merging PatentsView, DISCERN, and Clinical Trials
## Biopharma Firm's AI Capabilities via Patent Applications
#### Edward Jung

**Objective:** Construct a firm-year dataset of AI-related patent applications for firms conducting clinical trials.

**Key Differences from Task #1:**
- Focus on **patent applications** (not just granted patents)
- Use **g_application** table (captures earliest innovation timing)
- Map to **gvkey** using DISCERN 2 database
- Time period: **2000-2025**

---

## Table of Contents
1. [Data Architecture Design](#1-data-architecture-design)
2. [Data Import & Setup](#2-data-import--setup)
3. [PatentsView: Patent Applications (2000-2025)](#3-patentsview-patent-applications-2000-2025)
4. [DISCERN 2: Firm-to-GVKEY Mapping](#4-discern-2-firm-to-gvkey-mapping)
5. [AI Classification Logic](#5-ai-classification-logic)
6. [Firm-Year Aggregation](#6-firm-year-aggregation)
7. [Merge with Clinical Trials Dataset](#7-merge-with-clinical-trials-dataset)
8. [Export Final Datasets](#8-export-final-datasets)

---

## 1. Data Architecture Design

### Recommended Two-Layer Approach

#### Layer 1: **Patent-Level Dataset** (Intermediate)
One row per patent application

| Column | Type | Description |
|--------|------|-------------|
| application_id | str | Unique patent application identifier |
| patent_id | str | Patent ID if granted (may be null) |
| filing_date | date | Application filing date |
| filing_year | int | Year extracted from filing_date |
| assignee_name | str | Raw assignee/applicant name |
| gvkey | str | GVKEY from DISCERN 2 mapping |
| is_ai | bool | Binary: 1 if AI-related, 0 otherwise |
| ai_method | str | How AI was identified: 'cpc', 'keyword', or 'both' |
| ai_cpc_codes | str | Comma-separated AI CPC codes found |
| ai_keywords | str | AI keywords matched in title/abstract |
| title | str | Patent application title |
| abstract | str | Patent abstract text |

#### Layer 2: **Firm-Year Dataset** (Final Output)
One row per gvkey-year combination

| Column | Type | Description |
|--------|------|-------------|
| gvkey | str | Firm identifier from Compustat |
| year | int | Calendar year |
| total_applications | int | Total patent applications filed |
| ai_applications | int | AI-related applications |
| ai_share | float | ai_applications / total_applications |
| ai_dummy | int | 1 if ai_applications > 0, else 0 |

### Memory Efficiency Strategy

1. **Chunked Reading:** Process large TSV files in chunks (100k-500k rows)
2. **Categorical Types:** Convert repetitive strings (year, gvkey) to category dtype
3. **Early Filtering:** Filter to 2000-2025 and relevant firms before loading full data
4. **DuckDB Pre-filtering:** Use SQL to filter before pandas import
5. **Column Pruning:** Drop unnecessary columns immediately after import
6. **Incremental Processing:** Process year-by-year if memory constrained

### Why Two Layers?

- **Patent-level:** Allows validation, spot-checking, manual review
- **Firm-year:** Research-ready for regression analysis
- **Reproducibility:** Can regenerate firm-year from patent-level if needed
- **Flexibility:** Easy to add new metrics without re-processing raw data

## 2. Data Import & Setup

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import re
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# For large file processing
import duckdb
from zipfile import ZipFile
from urllib.request import urlretrieve

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

print("Setup complete!")
print(f"Working directory: {os.getcwd()}")

### Load Clinical Trials Dataset

In [ ]:
# Load clinical trials sample
clinical_trials = pd.read_csv('clinical_trial_sample (1).csv')

print(f"Clinical Trials Dataset Shape: {clinical_trials.shape}")
print(f"\nColumns: {clinical_trials.columns.tolist()}")
print(f"\nDate range: {clinical_trials['start_year'].min()} - {clinical_trials['start_year'].max()}")
print(f"\nUnique firms (gvkey): {clinical_trials['gvkey_sponsor'].nunique()}")
print(f"Unique NCT IDs: {clinical_trials['nct_id'].nunique()}")

# Display sample
clinical_trials.head()

In [ ]:
# Extract unique firms for filtering patent data
unique_gvkeys = clinical_trials['gvkey_sponsor'].dropna().unique()
unique_sponsors = clinical_trials['sponsor_name'].dropna().unique()

print(f"Number of unique GVKEYs: {len(unique_gvkeys)}")
print(f"Number of unique sponsor names: {len(unique_sponsors)}")
print(f"\nSample sponsor names:")
for sponsor in unique_sponsors[:10]:
    print(f"  - {sponsor}")

## 3. PatentsView: Patent Applications (2000-2025)

### Key PatentsView Tables for Applications

According to Task #2 requirements, we need **application-level** data:

1. **g_application** - Core application information
   - `application_id`: Unique identifier
   - `filing_date`: Application date (KEY for temporal alignment)
   - `patent_id`: Patent ID if granted (may be NULL for pending)
   
2. **pg_applicant_not_disambiguated** - Applicant names (for DISCERN matching)
   - `application_id`: Links to g_application
   - `applicant_organization`: Company name (raw, not disambiguated)
   
3. **g_cpc_current** - CPC classification codes (for AI identification)
   - Can link via `patent_id` (only for granted applications)
   
4. **g_us_application_citation** or **g_patent_abstract** - For keyword search

### Data Download Strategy

**Option A: Download from PatentsView bulk data**
- Base URL: https://s3.amazonaws.com/data.patentsview.org/download/
- Files: `g_application.tsv.zip`, `pg_applicant_not_disambiguated.tsv.zip`

**Option B: Use existing Task1 data + supplement**
- Task1 has granted patents (2021)
- Need to download application-specific tables

**Recommended: Option A** (complete application data)

In [ ]:
# Utility function for downloading PatentsView data
def download_patentsview_table(table_name, data_dir='../Task1', overwrite=False):
    """
    Download and extract PatentsView table.
    
    Parameters:
    -----------
    table_name : str
        Name of table (e.g., 'g_application')
    data_dir : str
        Directory to save files
    overwrite : bool
        Whether to re-download if file exists
    """
    base_url = "https://s3.amazonaws.com/data.patentsview.org/download"
    zip_url = f"{base_url}/{table_name}.tsv.zip"
    filename = f"{table_name}.tsv"
    filepath = Path(data_dir) / filename
    
    # Create directory if doesn't exist
    Path(data_dir).mkdir(parents=True, exist_ok=True)
    
    if filepath.exists() and not overwrite:
        print(f"✓ {filename} already exists")
        return str(filepath)
    
    print(f"Downloading {table_name}...")
    zip_path = filepath.with_suffix('.tsv.zip')
    
    try:
        # Download ZIP file
        urlretrieve(zip_url, zip_path)
        
        # Extract TSV
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        
        # Remove ZIP file
        zip_path.unlink()
        
        print(f"✓ Downloaded and extracted {filename}")
        return str(filepath)
        
    except Exception as e:
        print(f"✗ Error downloading {table_name}: {e}")
        return None

print("Download utility loaded.")

In [ ]:
# Download required tables (THIS MAY TAKE SEVERAL MINUTES)
required_tables = [
    'g_application',                    # Core application data
    'pg_applicant_not_disambiguated',   # Applicant names for matching
    'g_cpc_current',                    # Already downloaded in Task1
    'g_patent_abstract'                 # Already downloaded in Task1
]

print("Checking/downloading required PatentsView tables...\n")
for table in required_tables:
    download_patentsview_table(table)
    
print("\n✓ All required tables ready.")

### Memory-Efficient Import Using DuckDB

**Why DuckDB?**
- Handles multi-GB files without loading into memory
- SQL interface for filtering before pandas import
- Fast aggregations and joins
- No server setup required

In [ ]:
# Initialize DuckDB connection
con = duckdb.connect('task2_patents.ddb')

print("DuckDB initialized: task2_patents.ddb")
print(f"Database location: {os.path.abspath('task2_patents.ddb')}")

In [ ]:
# Import g_application table
print("Importing g_application (this may take 2-3 minutes)...")

con.execute("""
    CREATE OR REPLACE TABLE g_application AS 
    SELECT * FROM read_csv('../Task1/g_application.tsv', 
                           delim='\t', 
                           header=true,
                           all_varchar=true)
""")

# Check import
result = con.execute("SELECT COUNT(*) as total FROM g_application").fetchdf()
print(f"✓ Total applications loaded: {result['total'].iloc[0]:,}")

# Show sample
print("\nSample records:")
con.execute("SELECT * FROM g_application LIMIT 3").df()

In [ ]:
# Filter to 2000-2025 applications
print("Filtering to 2000-2025 application years...")

con.execute("""
    CREATE OR REPLACE VIEW applications_2000_2025 AS
    SELECT *,
           CAST(SUBSTRING(filing_date, 1, 4) AS INTEGER) as filing_year
    FROM g_application
    WHERE filing_date >= '2000-01-01' 
      AND filing_date <= '2025-12-31'
""")

# Count applications by year
yearly_counts = con.execute("""
    SELECT 
        filing_year,
        COUNT(*) as application_count
    FROM applications_2000_2025
    GROUP BY filing_year
    ORDER BY filing_year
""").df()

print(f"\nApplications by year (2000-2025):")
print(yearly_counts)
print(f"\nTotal applications (2000-2025): {yearly_counts['application_count'].sum():,}")

In [ ]:
# Import applicant names
print("Importing pg_applicant_not_disambiguated...")

con.execute("""
    CREATE OR REPLACE TABLE pg_applicant AS 
    SELECT * FROM read_csv('../Task1/pg_applicant_not_disambiguated.tsv', 
                           delim='\t', 
                           header=true,
                           all_varchar=true)
""")

result = con.execute("SELECT COUNT(*) as total FROM pg_applicant").fetchdf()
print(f"✓ Total applicant records: {result['total'].iloc[0]:,}")

# Show sample with organization names
print("\nSample applicant records:")
con.execute("""
    SELECT * FROM pg_applicant 
    WHERE applicant_organization IS NOT NULL 
    LIMIT 5
""").df()

## 4. DISCERN 2: Firm-to-GVKEY Mapping

### DISCERN 2 Overview

**Reference:** https://zenodo.org/records/13619821

DISCERN 2 provides:
- Mapping between patent assignees and Compustat GVKEY
- Handles name variations and disambiguation
- Time-varying firm identifiers

### Required DISCERN 2 Files

1. **Main mapping file:** Links assignee names → gvkey
2. **Time-varying mappings:** Tracks firm changes over time (mergers, acquisitions)

### Implementation Strategy

**Option A: Direct Name Matching**
- Match `pg_applicant.applicant_organization` to DISCERN assignee names
- Join on cleaned/standardized names

**Option B: Use Existing Clinical Trials Mapping**
- Clinical trials dataset already has sponsor_name → gvkey mapping
- Use this as ground truth for fuzzy matching to patent applicants

**Recommended: Hybrid Approach**
1. Start with clinical trials sponsor names
2. Match to patent applicant names using fuzzy matching
3. Validate with DISCERN 2 where available

### Placeholder: DISCERN 2 Integration

**Note:** DISCERN 2 data files are not included in this repository. 

**To integrate DISCERN 2:**
1. Download from https://zenodo.org/records/13619821
2. Extract relevant tables (consult data dictionary)
3. Load into DuckDB or pandas
4. Implement join logic below

**For now, we'll use the clinical trials dataset's existing gvkey mapping as a proxy.**

In [ ]:
# Create sponsor name → gvkey lookup from clinical trials
sponsor_gvkey_map = (
    clinical_trials[['sponsor_name', 'gvkey_sponsor']]
    .drop_duplicates()
    .dropna()
)

print(f"Sponsor-GVKEY mapping created: {len(sponsor_gvkey_map)} unique mappings")
print(f"\nSample mappings:")
sponsor_gvkey_map.head(10)

In [ ]:
# Helper function: Clean organization names for matching
def clean_org_name(name):
    """
    Standardize organization names for matching.
    
    Removes:
    - Legal suffixes (Inc., Corp., Ltd., etc.)
    - Punctuation
    - Extra whitespace
    
    Converts to lowercase.
    """
    if pd.isna(name):
        return ''
    
    name = str(name).lower()
    
    # Remove legal suffixes
    suffixes = [
        r'\binc\.?\b', r'\bcorp\.?\b', r'\bcorporation\b',
        r'\bltd\.?\b', r'\blimited\b', r'\bco\.?\b',
        r'\bllc\b', r'\blp\b', r'\bplc\b',
        r'\bsa\b', r'\bag\b', r'\bgmbh\b'
    ]
    for suffix in suffixes:
        name = re.sub(suffix, '', name)
    
    # Remove punctuation
    name = re.sub(r'[^a-z0-9\s]', ' ', name)
    
    # Remove extra whitespace
    name = ' '.join(name.split())
    
    return name.strip()

# Test cleaning function
test_names = [
    'Pfizer Inc.',
    'Bristol-Myers Squibb',
    'Eli Lilly and Company',
    'Novartis AG'
]

print("Name cleaning examples:")
for name in test_names:
    print(f"  {name:30s} → {clean_org_name(name)}")

In [ ]:
# Create cleaned name lookup
sponsor_gvkey_map['sponsor_name_clean'] = sponsor_gvkey_map['sponsor_name'].apply(clean_org_name)

# Remove duplicates after cleaning (some names may collapse to same cleaned version)
sponsor_lookup = (
    sponsor_gvkey_map
    .groupby('sponsor_name_clean')['gvkey_sponsor']
    .first()  # Take first gvkey if multiple matches
    .to_dict()
)

print(f"Cleaned sponsor lookup created: {len(sponsor_lookup)} unique clean names")
print(f"\nSample lookup:")
for i, (clean_name, gvkey) in enumerate(list(sponsor_lookup.items())[:10]):
    print(f"  {clean_name:30s} → {gvkey}")

### Match Patent Applicants to GVKEY

In [ ]:
# Extract applicants for applications in our time window
print("Extracting applicants for 2000-2025 applications...")

applicants_df = con.execute("""
    SELECT DISTINCT
        a.application_id,
        a.filing_date,
        a.filing_year,
        a.patent_id,
        p.applicant_organization
    FROM applications_2000_2025 a
    INNER JOIN pg_applicant p ON a.application_id = p.application_id
    WHERE p.applicant_organization IS NOT NULL
""").df()

print(f"✓ Extracted {len(applicants_df):,} application-applicant pairs")
print(f"\nSample:")
applicants_df.head()

In [ ]:
# Clean applicant names and match to gvkey
print("Matching applicant names to GVKEY...")

applicants_df['applicant_clean'] = applicants_df['applicant_organization'].apply(clean_org_name)
applicants_df['gvkey'] = applicants_df['applicant_clean'].map(sponsor_lookup)

# Check match rate
match_rate = (applicants_df['gvkey'].notna().sum() / len(applicants_df)) * 100
print(f"\nMatch rate: {match_rate:.2f}%")
print(f"Matched applications: {applicants_df['gvkey'].notna().sum():,}")
print(f"Unmatched applications: {applicants_df['gvkey'].isna().sum():,}")

# Filter to matched applications only
matched_applications = applicants_df[applicants_df['gvkey'].notna()].copy()
print(f"\n✓ Working with {len(matched_applications):,} matched applications")

matched_applications.head()

In [ ]:
# Check coverage: which firms from clinical trials have patent applications?
print("Firms with patent applications:")

firms_with_patents = matched_applications['gvkey'].unique()
firms_in_trials = clinical_trials['gvkey_sponsor'].dropna().unique()

coverage = (len(set(firms_with_patents) & set(firms_in_trials)) / len(firms_in_trials)) * 100

print(f"  Clinical trial firms: {len(firms_in_trials)}")
print(f"  Firms with patents: {len(firms_with_patents)}")
print(f"  Overlap: {len(set(firms_with_patents) & set(firms_in_trials))}")
print(f"  Coverage: {coverage:.1f}%")

## 5. AI Classification Logic

### Two-Pronged AI Identification Strategy

#### Method 1: CPC Classification Codes
**High Precision Approach**

AI-related CPC codes (from WIPO/EPO standards):
- **G06N** - Computing based on specific computational models
  - G06N3 - Neural networks
  - G06N5 - Knowledge-based models
  - G06N7 - Probabilistic/fuzzy logic
  - G06N10 - Quantum computing
  - G06N20 - Machine learning

**Advantages:**
- Examiner-assigned (authoritative)
- Standardized internationally
- High precision

**Limitations:**
- Only available for granted patents (not pending applications)
- May miss emerging AI applications not yet classified

#### Method 2: Keyword-Based Filtering
**High Recall Approach**

Search title/abstract for AI-related terms:
- Core ML: machine learning, deep learning, neural network, artificial intelligence
- Techniques: reinforcement learning, supervised learning, unsupervised learning
- Models: random forest, gradient boosting, support vector machine
- Applications: computer vision, natural language processing, NLP

**Advantages:**
- Works for both granted and pending applications
- Captures emerging terminology
- Higher recall

**Limitations:**
- May include false positives
- Requires careful keyword curation

### Recommended: Hybrid Approach
- **Primary:** CPC codes (for granted patents)
- **Secondary:** Keywords (for all applications, especially pending)
- **Combined:** Mark as AI if either method flags it

In [ ]:
# Define AI-related CPC code patterns
AI_CPC_PATTERNS = [
    'G06N3',   # Neural networks
    'G06N5',   # Knowledge-based models
    'G06N7',   # Probabilistic/fuzzy logic
    'G06N10',  # Quantum computing
    'G06N20',  # Machine learning
]

# Define AI-related keywords (lowercase)
AI_KEYWORDS = [
    # Core ML terms
    'machine learning', 'deep learning', 'neural network', 'artificial intelligence',
    'ai model', 'ml model', 'deep neural', 'convolutional neural',
    
    # Learning paradigms
    'supervised learning', 'unsupervised learning', 'reinforcement learning',
    'semi-supervised', 'transfer learning', 'meta-learning',
    
    # Specific models
    'random forest', 'gradient boosting', 'support vector machine', 'svm',
    'decision tree', 'bayesian network', 'recurrent neural', 'lstm',
    'transformer', 'attention mechanism', 'generative adversarial',
    
    # Applications
    'computer vision', 'natural language processing', 'nlp',
    'image recognition', 'speech recognition', 'predictive model',
    
    # Techniques
    'feature extraction', 'dimensionality reduction', 'classification algorithm',
    'regression algorithm', 'clustering algorithm'
]

print(f"AI CPC patterns defined: {len(AI_CPC_PATTERNS)}")
print(f"AI keywords defined: {len(AI_KEYWORDS)}")

In [ ]:
# Function: Check if text contains AI keywords
def contains_ai_keywords(text):
    """
    Check if text contains any AI-related keywords.
    
    Returns:
    --------
    tuple: (bool, list of matched keywords)
    """
    if pd.isna(text):
        return False, []
    
    text_lower = str(text).lower()
    matched_keywords = []
    
    for keyword in AI_KEYWORDS:
        if keyword in text_lower:
            matched_keywords.append(keyword)
    
    return len(matched_keywords) > 0, matched_keywords

# Test function
test_texts = [
    "A machine learning approach to drug discovery",
    "Novel pharmaceutical composition",
    "Deep neural network for protein folding prediction"
]

print("Keyword detection test:")
for text in test_texts:
    is_ai, keywords = contains_ai_keywords(text)
    print(f"  {text[:50]:50s} → AI: {is_ai}, Keywords: {keywords}")

### Classify Applications as AI-Related

In [ ]:
# Step 1: Get CPC codes for granted patents
print("Extracting CPC codes for matched applications...")

# Get patent IDs from matched applications (only granted ones have CPC codes)
granted_patent_ids = matched_applications['patent_id'].dropna().unique()

print(f"Granted patents in matched set: {len(granted_patent_ids):,}")

# Query CPC codes (using Task1 data)
cpc_codes_df = con.execute(f"""
    SELECT DISTINCT
        patent_id,
        cpc_group
    FROM g_cpc_current
    WHERE patent_id IN ({','.join("'" + str(pid) + "'" for pid in granted_patent_ids[:1000])})
""").df()

print(f"✓ Extracted CPC codes for {cpc_codes_df['patent_id'].nunique():,} patents")
cpc_codes_df.head()

In [ ]:
# Identify AI patents based on CPC codes
def is_ai_cpc(cpc_group):
    """Check if CPC code matches AI patterns."""
    if pd.isna(cpc_group):
        return False
    for pattern in AI_CPC_PATTERNS:
        if str(cpc_group).startswith(pattern):
            return True
    return False

cpc_codes_df['is_ai_cpc'] = cpc_codes_df['cpc_group'].apply(is_ai_cpc)

# Get AI patents by CPC
ai_patents_cpc = cpc_codes_df[cpc_codes_df['is_ai_cpc']]['patent_id'].unique()
print(f"AI patents identified by CPC: {len(ai_patents_cpc):,}")

# Get AI CPC codes found
ai_cpc_codes = (
    cpc_codes_df[cpc_codes_df['is_ai_cpc']]
    .groupby('patent_id')['cpc_group']
    .apply(lambda x: ','.join(x))
    .to_dict()
)

In [ ]:
# Step 2: Get titles/abstracts for keyword search
print("Loading patent titles and abstracts...")

# Get titles from g_application (if available) or g_patent
# For now, using Task1 g_patent table as proxy
titles_abstracts = con.execute("""
    SELECT 
        p.patent_id,
        p.patent_title as title,
        a.patent_abstract as abstract
    FROM g_patent p
    LEFT JOIN g_patent_abstract a ON p.patent_id = a.patent_id
""").df()

print(f"✓ Loaded titles/abstracts for {len(titles_abstracts):,} patents")
titles_abstracts.head()

In [ ]:
# Apply keyword detection
print("Detecting AI keywords in titles and abstracts...")

# Combine title and abstract for search
titles_abstracts['combined_text'] = (
    titles_abstracts['title'].fillna('') + ' ' + 
    titles_abstracts['abstract'].fillna('')
)

# Apply keyword detection (this may take a few minutes for large datasets)
keyword_results = titles_abstracts['combined_text'].apply(contains_ai_keywords)
titles_abstracts['is_ai_keyword'] = keyword_results.apply(lambda x: x[0])
titles_abstracts['ai_keywords_found'] = keyword_results.apply(lambda x: ','.join(x[1]))

ai_patents_keyword = titles_abstracts[titles_abstracts['is_ai_keyword']]['patent_id'].unique()
print(f"\n✓ AI patents identified by keywords: {len(ai_patents_keyword):,}")

# Show sample AI patents
print("\nSample AI patents identified by keywords:")
titles_abstracts[titles_abstracts['is_ai_keyword']][['patent_id', 'title', 'ai_keywords_found']].head()

In [ ]:
# Combine CPC and keyword classifications
print("Combining CPC and keyword classifications...")

# Merge back to matched_applications
matched_applications['is_ai_cpc'] = matched_applications['patent_id'].isin(ai_patents_cpc)
matched_applications = matched_applications.merge(
    titles_abstracts[['patent_id', 'is_ai_keyword', 'ai_keywords_found', 'title']],
    on='patent_id',
    how='left'
)

# Create combined AI flag
matched_applications['is_ai'] = (
    matched_applications['is_ai_cpc'] | 
    matched_applications['is_ai_keyword'].fillna(False)
)

# Add AI method indicator
def get_ai_method(row):
    if row['is_ai_cpc'] and row['is_ai_keyword']:
        return 'both'
    elif row['is_ai_cpc']:
        return 'cpc'
    elif row['is_ai_keyword']:
        return 'keyword'
    else:
        return None

matched_applications['ai_method'] = matched_applications.apply(get_ai_method, axis=1)

# Summary statistics
print(f"\n=== AI CLASSIFICATION SUMMARY ===")
print(f"Total applications: {len(matched_applications):,}")
print(f"AI by CPC only: {(matched_applications['ai_method'] == 'cpc').sum():,}")
print(f"AI by keyword only: {(matched_applications['ai_method'] == 'keyword').sum():,}")
print(f"AI by both methods: {(matched_applications['ai_method'] == 'both').sum():,}")
print(f"Total AI applications: {matched_applications['is_ai'].sum():,}")
print(f"AI share: {(matched_applications['is_ai'].sum() / len(matched_applications) * 100):.2f}%")

## 6. Firm-Year Aggregation

### Create Research-Ready Firm-Year Dataset

In [ ]:
# Aggregate to firm-year level
print("Aggregating to firm-year level...")

firm_year = (
    matched_applications
    .groupby(['gvkey', 'filing_year'])
    .agg({
        'application_id': 'count',              # Total applications
        'is_ai': 'sum'                          # AI applications
    })
    .rename(columns={
        'application_id': 'total_applications',
        'is_ai': 'ai_applications'
    })
    .reset_index()
)

# Calculate derived metrics
firm_year['ai_share'] = firm_year['ai_applications'] / firm_year['total_applications']
firm_year['ai_dummy'] = (firm_year['ai_applications'] > 0).astype(int)

# Rename filing_year to year for clarity
firm_year = firm_year.rename(columns={'filing_year': 'year'})

print(f"✓ Firm-year dataset created: {len(firm_year):,} observations")
print(f"  Unique firms: {firm_year['gvkey'].nunique()}")
print(f"  Year range: {firm_year['year'].min()} - {firm_year['year'].max()}")

firm_year.head(10)

In [ ]:
# Summary statistics
print("=== FIRM-YEAR DATASET SUMMARY ===")
print(f"\nSample size: {len(firm_year):,} firm-year observations")
print(f"\nApplications per firm-year:")
print(firm_year['total_applications'].describe())
print(f"\nAI applications per firm-year:")
print(firm_year['ai_applications'].describe())
print(f"\nAI share distribution:")
print(firm_year['ai_share'].describe())
print(f"\nFirm-years with at least one AI patent: {firm_year['ai_dummy'].sum():,} ({firm_year['ai_dummy'].mean()*100:.1f}%)")

In [ ]:
# Temporal trends: AI adoption over time
yearly_trends = (
    firm_year
    .groupby('year')
    .agg({
        'total_applications': 'sum',
        'ai_applications': 'sum',
        'ai_dummy': 'sum',  # Number of firms with AI
        'gvkey': 'nunique'  # Number of firms
    })
    .rename(columns={'gvkey': 'num_firms', 'ai_dummy': 'firms_with_ai'})
)

yearly_trends['ai_share'] = yearly_trends['ai_applications'] / yearly_trends['total_applications']
yearly_trends['firm_adoption_rate'] = yearly_trends['firms_with_ai'] / yearly_trends['num_firms']

print("\n=== TEMPORAL TRENDS ===")
print(yearly_trends)

## 7. Merge with Clinical Trials Dataset

In [ ]:
# Create clinical trials firm-year dataset
print("Creating clinical trials firm-year dataset...")

trials_firm_year = (
    clinical_trials
    .groupby(['gvkey_sponsor', 'start_year'])
    .agg({
        'nct_id': 'count',
        'phase_number': 'mean'  # Average phase
    })
    .rename(columns={
        'nct_id': 'num_trials',
        'phase_number': 'avg_phase'
    })
    .reset_index()
    .rename(columns={'gvkey_sponsor': 'gvkey', 'start_year': 'year'})
)

print(f"✓ Clinical trials firm-year: {len(trials_firm_year):,} observations")
trials_firm_year.head()

In [ ]:
# Merge patent and trial datasets
print("Merging patent applications with clinical trials...")

merged_firm_year = firm_year.merge(
    trials_firm_year,
    on=['gvkey', 'year'],
    how='outer',  # Keep all firm-years from both datasets
    indicator=True
)

# Fill NAs with 0 for count variables
count_vars = ['total_applications', 'ai_applications', 'ai_dummy', 'num_trials']
for var in count_vars:
    merged_firm_year[var] = merged_firm_year[var].fillna(0).astype(int)

# Recalculate ai_share
merged_firm_year['ai_share'] = np.where(
    merged_firm_year['total_applications'] > 0,
    merged_firm_year['ai_applications'] / merged_firm_year['total_applications'],
    0
)

print(f"\n✓ Merged dataset: {len(merged_firm_year):,} firm-year observations")
print(f"\nMerge statistics:")
print(merged_firm_year['_merge'].value_counts())

# Drop merge indicator
merged_firm_year = merged_firm_year.drop('_merge', axis=1)

merged_firm_year.head(10)

In [ ]:
# Summary of merged dataset
print("=== MERGED FIRM-YEAR DATASET SUMMARY ===")
print(f"\nTotal observations: {len(merged_firm_year):,}")
print(f"Unique firms: {merged_firm_year['gvkey'].nunique()}")
print(f"Year range: {merged_firm_year['year'].min()} - {merged_firm_year['year'].max()}")
print(f"\nFirm-years with patents: {(merged_firm_year['total_applications'] > 0).sum():,}")
print(f"Firm-years with AI patents: {(merged_firm_year['ai_applications'] > 0).sum():,}")
print(f"Firm-years with trials: {(merged_firm_year['num_trials'] > 0).sum():,}")
print(f"Firm-years with both patents and trials: {((merged_firm_year['total_applications'] > 0) & (merged_firm_year['num_trials'] > 0)).sum():,}")

## 8. Export Final Datasets

In [ ]:
# Export patent-level dataset
patent_level_output = matched_applications[[
    'application_id', 'patent_id', 'filing_date', 'filing_year',
    'gvkey', 'applicant_organization', 'applicant_clean',
    'is_ai', 'ai_method', 'is_ai_cpc', 'is_ai_keyword', 'ai_keywords_found',
    'title'
]].copy()

patent_level_output.to_csv('patent_level_dataset.csv', index=False)
print(f"✓ Exported patent-level dataset: patent_level_dataset.csv")
print(f"  Shape: {patent_level_output.shape}")

In [ ]:
# Export firm-year dataset (patents only)
firm_year.to_csv('firm_year_patents.csv', index=False)
print(f"✓ Exported firm-year patent dataset: firm_year_patents.csv")
print(f"  Shape: {firm_year.shape}")

In [ ]:
# Export merged firm-year dataset (patents + trials)
merged_firm_year.to_csv('firm_year_merged.csv', index=False)
print(f"✓ Exported merged firm-year dataset: firm_year_merged.csv")
print(f"  Shape: {merged_firm_year.shape}")

## Summary & Next Steps

### Deliverables Created

1. **patent_level_dataset.csv**
   - One row per patent application
   - Contains AI classification flags and methods
   - ~{patent_level_output.shape[0]:,} applications

2. **firm_year_patents.csv**
   - One row per gvkey-year
   - Patent application metrics: total, AI count, AI share
   - ~{firm_year.shape[0]:,} firm-year observations

3. **firm_year_merged.csv**
   - Combined patent applications + clinical trials
   - Ready for regression analysis
   - ~{merged_firm_year.shape[0]:,} firm-year observations

### Key Findings

- **Match Rate:** {match_rate:.1f}% of patent applications matched to clinical trial firms
- **AI Patents:** {matched_applications['is_ai'].sum():,} AI-related applications identified
- **AI Share:** {(matched_applications['is_ai'].mean()*100):.2f}% of applications are AI-related
- **Temporal Coverage:** 2000-2025

### Recommended Next Steps

1. **DISCERN 2 Integration**
   - Download DISCERN 2 database
   - Improve gvkey matching coverage
   - Handle time-varying firm identifiers (M&A)

2. **Validation**
   - Manual review of AI classification accuracy
   - Compare to known AI patents/firms
   - Refine keyword list based on false positives

3. **Extended Analysis**
   - Lag structures (patents → trials)
   - Firm-specific AI intensity trends
   - Technology subfield analysis (drug discovery vs. clinical trial AI)

4. **PubMed Linkage** (Task #2 Part 2)
   - Implement NCT ID → PubMed search
   - Identify AI methods in trial publications

### Memory Efficiency Notes

**For larger datasets:**
1. Process year-by-year in chunks
2. Use DuckDB for all filtering/aggregation
3. Keep only necessary columns in memory
4. Use categorical dtypes for string columns
5. Consider Dask/Vaex for very large datasets (>50GB)